# <b>■ 강화학습</b>
    
    인공지능 -----> 머신러닝
    1. 지도학습 : knn, 의사결정 나무, 서포트 벡터 머신, 나이브 베이즈, 신경망 
    2. 비지도 학습 : k-means, 아프리오리(연관분석) 
    3. 강화학습
        강화학습이 세상에 크게 알려지게된 계기? 알파고와 이세돌과의 바둑대결
        
        인공지능에 관심 ----> 신경망
        강화학습을 구현한 대표적인 회사 ---> 딥마인드
            데미스 하사비스가 2011년애 창업한 영국의 스타트업 회사 강화학습 핑퐁게임 
                --> 벽돌깨기 게임 을 구현하고 구글에 4300억원 ~ 5000억원에 인수 됨.
        딥마인드 인수를 페이스북에서 진행하다 실패해서 구글이 인수를 함
        
    강화학습의 개요
        '강화'는 동물이 시행착오를 통해 학습하는 방법중 하나이다.
        '강화'라는 개념을 처음에 제시한것은 스키너라는 행동 심리학자이다.
        행동 심리학에서는 시행착오 학습이라는 개념이 있다.
        시행착오 학습은 동물들이 이것저것 시도해 보면서 그 결과 환경을 학습하는 것을 말한다.
        대표적인게 스키너의 쥐 실험을 통해서 동물이 행동과 그 결과 사이의 관계를 학습 하는것을 확인했다.
        '강화' 라는 것은 동물이 이전에 배우지 않았지만 직접 시도하면서 행동과 그 결과로 나타나는 좋은 보상사이의 상관관계를 학습하는것이다. 
        그러면서 동물이 좋은 보상을 얻게 해주는 행동을 점점 더 많이 하는것을 말한다.
        "강화의 핵심은 바로 보상을 얻게 해주는 행동의 빈도 증가이다."
    

####    강화학습의 결과를 실습 : 인공지능 핑퐁 코드를 구현하시오

In [ ]:
#######################################################################################
# Game : 1인 핑퐁 게임 머신 러닝 코드
#######################################################################################

from tkinter import *
import random
import time
import csv

EMPTY = 0
PADDLE_HEIGHT = 360.0
PADDLE_MOVE = [-8, 8]   # PADDLE이 좌우로 이동하는 속도

START = 3
END = 4

class Ball:
    def __init__(self, canvas, paddle, color, announceterm, saveterm, winval = 10, loseval = -1):
        self.canvas = canvas
        self.paddle = paddle
        self.id = canvas.create_oval(10, 10, 25, 25, fill=color)  # 공 크기 및 색깔

        self.canvas.move(self.id, 245, 200)  # 공을 캔버스 중앙으로 이동
        starts = [-3, 3]  # 공의 속도를 랜덤으로 구성하기 위해 준비한 리스트
        random.shuffle(starts)  # starts 리스트 중에 숫자를 랜덤으로 골라서
        self.x = starts[0]  # 처음 공이 패들에서 움직일때 왼쪽으로 올라갈지 오른쪽으로 올라갈지 랜덤으로 결정되는 부분
        self.y = -3  # 처음 공이 패들에서 움직일때 위로 올라가는 속도
        self.canvas_height = self.canvas.winfo_height()  # 캔버스의 현재 높이를 반환한다.(공이 화면에서 사라지지 않기위해)
        self.canvas_width = self.canvas.winfo_width()  # 캔버스의 현재 넓이를 반환한다.(공이 화면에서 사라지지 않기위해)
        self.hit_bottom = False
        self.values = {}
        self.epsilon = 0 #랜덤율
        self.alpha = 0.99 #망각계수
        self.learning = True
        self.cycle_data = []
        self.wincount = 0
        self.losecount = 0
        self.gamecount = 0
        self.winval = winval #성공 보상치
        self.loseval = loseval #실패 보상치
        self.announceterm = announceterm # 게임 횟수 프린트 텀
        self.saveterm = saveterm # csv 저장 텀

        # csv 파일에서 gamecount / values 불러옴
        self.loadcsv()

    def action(self):
        r = random.random()
        if r < self.epsilon:
            direction = self.randomChoice()
        else:
            direction = self.greedyChoice()
        x = PADDLE_MOVE[direction]
        # 머신러닝을 위한 한 사이클 내 이동 데이터 저장
        self.cycle_data.append(self.keystate(direction))
        #이동/그리기
        self.paddle.move(x)
        self.paddle.draw()

    #이동 방향 랜덤으로 지정
    def randomChoice(self):
        rand = random.choice([0,1])
        key = self.keystate(rand)
        if key not in self.values:
            self.add(key)
        return rand

    #이동 방향 Greedy로 지정
    def greedyChoice(self):
        val_left = self.keystate(0)
        val_right = self.keystate(1)

        if self.lookup(val_left) > self.lookup(val_right):
            return 0
        elif self.lookup(val_left) < self.lookup(val_right):
            return 1
        else:
            return random.choice([0,1])

    def add(self, key):
        self.values[key] = 0

    def lookup(self, key):
        if key not in self.values:
            #print(key)
            self.add(key)
        return self.values[key]

    def hit_paddle(self, pos):  # 패들에 공이 튀기게 하는 함수
        paddle_pos = self.canvas.coords(self.paddle.id)
        # 공의 x좌표가 패들의 너비 안에 있는지 / 공 바닥이 패들 윗면에 닿아 있는지
        if pos[2] >= paddle_pos[0] and pos[0] <= paddle_pos[2] \
                and pos[3] == PADDLE_HEIGHT:
            return True
        return False

    #공을 패들로 받았는지 여부 출력(True/False)
    def is_paddle_hit(self):
        return self.hit_paddle(self.canvas.coords(self.id))

    def draw(self):
        # 볼의 현재 좌표를 출력해준다. 공 좌표( 좌상단 x,y좌표 / 우하단 x,y좌표 )
        pos = self.canvas.coords(self.id)
        # [ 255,29,270,44]

        if pos[1] <= 0:
            self.y = 3
        if pos[3] >= self.canvas_height:
            self.y = -3
        if pos[0] <= 0:
            self.x = 3
        if pos[2] >= self.canvas_width:
            self.x = -3
        if self.hit_paddle(pos) == True:
            self.y = -3

        self.canvas.move(self.id, self.x, self.y)  # 공을 움직이게 하는 부분
        # 공이 화면 밖으로 나가지 않게 해준다

    def keystate(self,movement):
        paddle_pos = self.canvas.coords(self.paddle.id)
        ball_pos = self.canvas.coords(self.id)
        #paddle 위치(좌측 x좌표), 공의 좌상단 x/y좌표, 공의 좌우/상하 속도(방향), paddle을 좌/우 중 어느 쪽으로 움직이는지
        return (paddle_pos[0],(ball_pos[0],ball_pos[1]),(self.x,self.y),movement)


    # 사이클 시작 : 1, 사이클 종료 : -1, 해당 없음 : 0
    def cyclestate(self):
        pos = self.canvas.coords(self.id)
        if pos[3] == PADDLE_HEIGHT:
            if self.y == -3:
                return START
            elif self.y == 3:
                return END
        return 0

    # 결과 학습
    def backup(self, newVal, idx):
        if idx >= 0 and self.learning:
            prevVal = self.values[self.cycle_data[idx]]
            self.values[self.cycle_data[idx]] += self.alpha * (newVal - prevVal)
            #print("key : {0}, val : {1}".format(self.cycle_data[idx],self.values[self.cycle_data[idx]]))
            self.backup(newVal*self.alpha,idx-1)

    # 게임 끝났을 시 학습 및 cycle_data 초기화
    def gameover(self):
        if self.learning:
            #paddle로 받았으면 1, 못 받았으면 -1
            if self.is_paddle_hit():
                result_value = self.winval
                self.wincount += 1
            else:
                result_value = self.loseval
                self.losecount += 1
            self.backup(result_value, len(self.cycle_data)-1)
            self.gamecount += 1

            #saveterm마다 csv 저장
            if self.gamecount % self.saveterm == 0:
                self.writecsv()
            if self.gamecount % self.announceterm == 0:
                print("cycle count : {0}".format(ball.gamecount))
        self.cycle_data.clear()

    def winnerval(self, winner):
        if winner == 'hit':
            return 1
        elif winner == 'miss':
            return -1
        else:
            return 0

    #게임 결과 csv로 저장
    def writecsv(self):
        try:
            # Values 저장
            Fn = open("D:\pong_value.csv", 'w', newline='')
            writer = csv.writer(Fn, delimiter=',')
            writer.writerow([self.gamecount]) #첫줄에 학습 게임 횟수 저장
            keys = self.values.keys()
            for key in keys:
                writer.writerow([key[0],
                            key[1][0],
                            key[1][1],
                            key[2][0],
                            key[2][1],
                            key[3],
                            ball.values[key]
                            ])
            Fn.close()

            #성공/실패 횟수 저장
            Fn = open("D:\pong_score.csv", 'a', newline='')
            writer = csv.writer(Fn, delimiter=',')
            writer.writerow([self.wincount, self.losecount, self.gamecount])
            Fn.close()
            #승률의 변화를 확인하기 위해 일정 판수마다 카운트 리셋
            self.wincount = 0
            self.losecount = 0

            print("save data in cycle {0}.".format(self.gamecount))
        except Exception as e:
            print('save data failed in cycle {0}.\nError Type : {1}'.format(self.gamecount,type(e).__name__))

    def loadcsv(self):
        try:
            Fn = open("D:\pong_value.csv", 'r')
            self.gamecount = int(Fn.readline().split(',')[0]) # 첫 줄의 학습 게임 횟수 불러오기
            reader = csv.reader(Fn, delimiter=',')
            for key in reader:
                self.values[(int(float(key[0])), (int(float(key[1])), int(float(key[2]))), (int(float(key[3])), int(float(key[4]))), int(float(key[5])))] = float(key[6])
            print('Load Success! Start at cycle {0}'.format(self.gamecount))
        except Exception:
            print('Load Failed!')

class Paddle:
    def __init__(self, canvas, y_loc, color):
        self.canvas = canvas
        self.id = canvas.create_rectangle(0, 0, 100, 10, fill=color)  # 패들의 높이와 넓이 그리고 색깔

        self.canvas.move(self.id, 200, y_loc)  # 패들 사각형을 200,300 에 위치
        self.x = 0  # 패들이 처음 시작할때 움직이지 않게 0으로 설정
        self.canvas_width = self.canvas.winfo_width()  # 캔버스의 넓이를 반환한다. 캔버스 밖으로 패들이 나가지 않도록
        self.canvas.bind_all('<KeyPress-Left>', self.turn_left)  # 왼쪽 화살표 키를 '<KeyPress-Left>'  라는 이름로 바인딩
        self.canvas.bind_all('<KeyPress-Right>', self.turn_right)  # 오른쪽도 마찬가지로 바인딩한다.

    def draw(self):
        pos = self.canvas.coords(self.id)
        # print(pos)
        if pos[0] <= 0 and self.x < 0:  # 패들의 위치가 왼쪽 끝이고, 이동하려는 방향이 왼쪽이면 함수 종료(이동 안 함)
            return
        elif pos[2] >= self.canvas_width and self.x > 0:  # 패들의 위치가 오른쪽 끝이고, 이동하려는 방향이 오른쪽이면 함수 종료
            return
        self.canvas.move(self.id, self.x, 0)


        # 패들이 화면의 끝에 부딪히면 공처럼 튕기는게 아니라 움직임이 멈춰야한다.
        # 그래서 왼쪽 x 좌표(pos[0]) 가 0 과 같거나 작으면 self.x = 0 처럼 x 변수에 0 을
        # 설정한다.  같은 방법으로 오른쪽 x 좌표(pos[2]) 가 캔버스의 폭과 같거나 크면
        # self.x = 0 처럼 변수에 0 을 설정한다.

    def turn_left(self, evt):  # 패들의 방향을 전환하는 함수
        self.x = -3

    def turn_right(self, evt):
        self.x = 3

    def move(self, x):
        self.x = x


if __name__ == '__main__':
    tk = Tk()  # tk 를 인스턴스화 한다.
    tk.title("Game")  # tk 객체의 title 메소드(함수)로 게임창에 제목을 부여한다.
    tk.resizable(0, 0)  # 게임창의 크기는 가로나 세로로 변경될수 없다라고 말하는것이다.
    tk.wm_attributes("-topmost", 1)  # 다른 모든 창들 앞에 캔버스를 가진 창이 위치할것을 tkinter 에게 알려준다.

    canvas = Canvas(tk, width=500, height=450, bd=0, highlightthickness=0)
    # bg=0,highlightthickness=0 은 캔버스 외곽에 둘러싼
    # 외곽선이 없도록 하는것이다. (게임화면이 좀더 좋게)
    canvas.pack()  # 앞의 코드에서 전달된 폭과 높이는 매개변수에 따라 크기를 맞추라고 캔버스에에 말해준다.
    tk.update()  # tkinter 에게 게임에서의 애니메이션을 위해 자신을 초기화하라고 알려주는것이다.
    paddle = Paddle(canvas, PADDLE_HEIGHT, 'blue')

    #announceterm : 현재 count 출력 term, saveterm : csv에 저장하는 term
    ball = Ball(canvas, paddle, 'red', announceterm=500, saveterm=500)
    start = False
    # 공을 약간 움직이고 새로운 위치로 화면을 다시 그리며, 잠깐 잠들었다가 다시 시작해 ! "
    is_cycling = False

    while 1:
        ball.draw()

        c_state = ball.cyclestate()
        if c_state == END:
            #print('END')
            ball.gameover()
            is_cycling = False
        if c_state == START or ball.is_paddle_hit():
            #print('START')
            is_cycling = True

        if is_cycling:
            ball.action()

        tk.update_idletasks()  # 우리가 창을 닫으라고 할때까지 계속해서 tkinter 에게 화면을 그려라 !
        tk.update()  # tkinter 에게 게임에서의 애니메이션을 위해 자신을 초기화하라고 알려주는것이다.

        #1만번 학습 후에 정상 속도로 플레이 시작(학습 결과 반영됨)
        if ball.gamecount > 10000:
            time.sleep(0.005)  # 무한 루프중에 100분의 1초마다 잠들어라 !
